In [8]:
import pandas as pd
import numpy as np

# Load the data and pivot so that each PID is a column
data = pd.read_csv('classified_driving_data.csv')
data_wide = data.pivot_table(index='interval', columns='PID', values='VALUE', aggfunc='mean')

Preprocessing data

In [9]:
from tensorflow import keras
from keras import Sequential

# Merge the classification column back to the df
classification = data[['interval', 'classification']].drop_duplicates()
data_wide = pd.merge(data_wide, classification, on='interval')

# Interval is not needed for training, and can be dropped here
X = data_wide.drop(columns=['classification', 'interval'])
y = keras.utils.to_categorical(data_wide['classification'].map({'Normal': 0, 'Slow': 1, 'Aggressive': 2}))

# Handle any missing values by filling with the mean of that column
X.fillna(X.mean(), inplace=True)

Preparing data for model training

In [10]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Split the data and scale features
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=42)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

Logistic Regression Model

In [11]:
from sklearn.linear_model import LogisticRegression

# Training the model
logreg_direct = LogisticRegression()
y_train_classes = np.argmax(y_train, axis=1)
logreg_direct.fit(X_train, y_train_classes)

# Prediction
y_pred_logreg_direct = logreg_direct.predict(X_test)
y_test_classes = np.argmax(y_test, axis=1)

Neural Network Model

In [12]:
model = Sequential()
model.add(keras.layers.Dense(64, input_dim=X_train.shape[1], activation='relu'))
model.add(keras.layers.Dense(32, activation='relu'))
model.add(keras.layers.Dense(16, activation='relu'))
model.add(keras.layers.Dense(3, activation='softmax')) 

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Training the model
model.fit(X_train, y_train, epochs=50, batch_size=32, validation_split=0.2)

# Predictions
y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_test_classes = np.argmax(y_test, axis=1)

Epoch 1/50


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.4200 - loss: 1.0390 - val_accuracy: 0.6516 - val_loss: 0.9067
Epoch 2/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 873us/step - accuracy: 0.6978 - loss: 0.8813 - val_accuracy: 0.7677 - val_loss: 0.7445
Epoch 3/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 814us/step - accuracy: 0.7886 - loss: 0.7138 - val_accuracy: 0.8452 - val_loss: 0.5769
Epoch 4/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 818us/step - accuracy: 0.8566 - loss: 0.5350 - val_accuracy: 0.9161 - val_loss: 0.4300
Epoch 5/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 810us/step - accuracy: 0.8511 - loss: 0.4155 - val_accuracy: 0.9226 - val_loss: 0.3269
Epoch 6/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 832us/step - accuracy: 0.9021 - loss: 0.3132 - val_accuracy: 0.9290 - val_loss: 0.2587
Epoch 7/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 870us/step - accuracy: 0.9124 - loss: 0.2514 - val_accuracy: 0.9419 - val_loss: 0.2149
Epoch 8/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 860us/step - accuracy: 0.9336 - loss: 0.2129 - val_accuracy: 0.9355 - val_lo

Model Results

In [13]:
from sklearn.metrics import classification_report, accuracy_score

print("Logistic Regression Accuracy:", accuracy_score(y_test_classes, y_pred_logreg_direct))
print("Logistic Regression Classification Report:\n", classification_report(y_test_classes, y_pred_logreg_direct))

print("Neural Network Accuracy:", accuracy_score(y_test_classes, y_pred_classes))
print("Classification Report:\n", classification_report(y_test_classes, y_pred_classes))

# Print what the majority of the training data was classified as
majority_class_index = np.argmax(np.sum(y_train, axis=0))
class_mapping = {0: 'normal', 1: 'slow', 2: 'aggressive'}
print("Based on the training data, the sample is generally classified as", class_mapping[majority_class_index], "driving.")

Logistic Regression Accuracy: 0.9786821705426356
Logistic Regression Classification Report:
               precision    recall  f1-score   support

           0       0.97      1.00      0.98       294
           1       1.00      0.96      0.98       119
           2       0.99      0.95      0.97       103

    accuracy                           0.98       516
   macro avg       0.99      0.97      0.98       516
weighted avg       0.98      0.98      0.98       516

Neural Network Accuracy: 0.9709302325581395
Classification Report:
               precision    recall  f1-score   support

           0       0.96      0.99      0.97       294
           1       1.00      0.93      0.97       119
           2       0.97      0.96      0.97       103

    accuracy                           0.97       516
   macro avg       0.98      0.96      0.97       516
weighted avg       0.97      0.97      0.97       516

Based on the training data, the sample is generally classified as normal driv